In [1]:
%websocketconnect --password wpass ws://10.0.37.237:8266/

** WebSocket connected **
Password: 
WebREPL connected
>>> Ready.


In [ ]:
import network
w = network.WLAN()
w.active(True)
w.connect("BV6000", "bubblino")

In [12]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [2]:
%sendtofile config.txt

wifiname        BV6000
wifipassword    bubblino
mqttbroker      192.168.43.1
pinneopixel     15
pinled          2
boardname       neogrid1
wifialt         DoESLiverpool,decafbad00,10.0.100.1

Sent 7 lines (190 bytes) to config.txt.


In [30]:
%sendtofile config.txt

wifiname        DoESLiverpool
wifipassword    decafbad00
mqttbroker      10.0.100.1
pinneopixel     15
pinled          2
boardname       neogrid1


Sent 6 lines (146 bytes) to config.txt.


In [18]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [19]:
%sendtofile main.py

from mqtt_as import MQTTClient, config
import uasyncio as asyncio
import os, time, machine

fconfig = dict(x.split()  for x in open("config.txt"))
pinled = machine.Pin(int(fconfig["pinled"]), machine.Pin.OUT)
 
import neopixel
np = neopixel.NeoPixel(machine.Pin(int(fconfig["pinneopixel"])), 81)
np.fill((20,0,80))
np.write()

import framebuf
fbuff = framebuf.FrameBuffer(bytearray(9*9*2), 9, 9, framebuf.RGB565)

def chex(su):
    u = int(su, 16)
    return (((u&0xF80000)>>8) | ((u&0x00FC00)>>5) | ((u&0x0000F8)>>3))

def show():
    for iy in range(9):
        oiy = (iy%2)
        fix = (-1 if oiy else 1)
        riy = iy*9 + 8*oiy
        for ix in range(9):
            v = fbuff.pixel(ix, iy)
            r = (v&0b1111100000000000)>>8
            g = (v&0b0000011111100000)>>2
            b = (v&0b0000000000011111)<<3
            np[riy + ix*fix] = (r, g, b) 
    np.write()
    
pinled.value(0)
time.sleep(2)
pinled.value(1)

boardname = fconfig["boardname"]

async def connectionmonitortask():
    t0 = time.time()
    while True:
        t = time.time()
        if client.isconnected():
            t0 = t
            aloop.create_task(client.publish(boardname+'/time', str(time.time())))
        elif (t - t0) > 60:
            fbuff.fill(0x0000); fbuff.text(str(int((t-t0)/60)), 0, 1, 0xf000); show()
            if t - t0 > 60*9:
                machine.reset()
        await asyncio.sleep_ms(20000)

async def callbackcmdtask(topic, msg):
    print("executing", [msg])
    pinled.value(0)
    try:
        exec(msg)
        await client.publish(boardname+'/reply', "1")
    except Exception as e:
        print(e)
        await client.publish(boardname+'/reply/exception', str(e))
    pinled.value(1)
    return

def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(topic, msg))
    
async def onconnecttask(client):
    print("subscribing")
    fbuff.fill(0x0000); fbuff.text("S", 0, 1, 0xf000); show()
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(boardname+'/ip', ipnumber, retain=True, qos=1)
    tboardcmd = boardname+'/cmd'
    print("subscribing to", tboardcmd)
    await client.subscribe(tboardcmd, 1)
    fbuff.fill(0x0000); fbuff.text("R", 0, 1, 0xf000); show()
    
async def mqtttask(bflip=False):
    print("mqttttask", bflip)
    if bflip and "wifialt" in fconfig:
        assid, awifi_pw, aserver = fconfig["wifialt"].split(",")
        if client._ssid == assid:
            assid, awifi_pw, aserver = fconfig["wifiname"], fconfig["wifipassword"], fconfig["mqttbroker"]
        client._ssid, client._wifi_pw, client.server = assid, awifi_pw, aserver
    fbuff.fill(0x0000); fbuff.text("D" if bflip else "C", 0, 1, 0xf000); show()
    try:
        await client.connect()
        return
    except OSError as e:
        print("client connect fail", [e])
    aloop.create_task(mqtttask(True))
        
        
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]

MQTTClient.DEBUG = True
client = MQTTClient(config)

aloop = asyncio.get_event_loop()
aloop.create_task(mqtttask())
aloop.create_task(connectionmonitortask())
aloop.run_forever()


Sent 106 lines (3291 bytes) to main.py.


In [16]:
%serialconnect

Connecting to --port=/dev/ttyUSB6 --baud=115200 
Ready.


In [ ]:
%rebootdevice

In [ ]:
#      RST          |  GPIO1   TX
#       A0          |  GPIO3   RX
#       D0  GPIO16  |  GPIO5   D1  SCL
# SCK   D5  GPIO14  |  GPIO4   D2  SDA
# MISO  D6  GPIO12  |  GPIO0   D3
# MOSI  D7  GPIO13  |  GPIO2   D4  LED
# SS    D8  GPIO15  |          GND
#      3V3          |          5V


In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [2]:
%sendtofile config.txt

wifiname        DoESLiverpool
wifipassword    decafbad00
mqttbroker      10.0.100.1
pinneopixel     15
pinled          2
boardname       neoring1


Sent 6 lines (146 bytes) to config.txt.


In [3]:
%sendtofile main.py

from mqtt_as import MQTTClient, config
import uasyncio as asyncio
import os, time, machine

fconfig = dict(x.split()  for x in open("config.txt"))
pinled = machine.Pin(int(fconfig["pinled"]), machine.Pin.OUT)
 
import neopixel
np = neopixel.NeoPixel(machine.Pin(int(fconfig["pinneopixel"])), 24)
np.fill((2,1,12))
np.write()

pinled.value(0)
time.sleep(2)
pinled.value(1)

boardname = fconfig["boardname"]
async def callbackcmdtask(topic, msg):
    print("executing", [msg])
    pinled.value(0)
    try:
        exec(msg)
        await client.publish(boardname+'/reply', "1")
    except Exception as e:
        print(e)
        await client.publish(boardname+'/reply/exception', str(e))
    pinled.value(1)
    return

def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(topic, msg))

async def mainconnect():
    print("connecting")
    np.fill((5,0,0)); np.write()
    await client.connect()
    print("subscribing")
    np.fill((0,5,0)); np.write()
    await client.subscribe(boardname+'/cmd', 1)
    await client.publish(boardname+"/ip", client._sta_if.ifconfig()[0], retain=True, qos=1)
    np.fill((2,1,12)); np.write()
    while True:
        await client.publish(boardname+'/time', str(time.time()))
        await asyncio.sleep_ms(2000)
    
config['subs_cb'] = callbackcmd
config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]

MQTTClient.DEBUG = True
client = MQTTClient(config)

aloop = asyncio.get_event_loop()
aloop.run_until_complete(mainconnect())


Sent 55 lines (1556 bytes) to main.py.


In [18]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [6]:
os.rename("main.py", "Dmain.py")

In [19]:
%sendtofile main.py

from OLED_driver import i2c, fbuff, oledshow, doublepixels, fatntext

print("fbuff commands", dir(fbuff))
print("commands are: oledshow, doublepixels, fatntext")

import time
for i in range(10):
    fbuff.fill(0)
    fbuff.text("Hi", i*2, i, 1)
    fbuff.text("there", i*2, i+8, 1)
    doublepixels()
    oledshow()
    time.sleep_ms(100)
oledshow()

for l in open("main.py"):
    print(l, end="")


Sent 17 lines (398 bytes) to main.py.
